This Notebook is used for two purposes. 

1. Once 2 (or 3*) members of the team have gone through the LSDCat outputs of a single field, this notebook can be used to make a master list of conflicting sources to be assesed by an independent person. This will make a catalogue similar to LSDCat format, so it can be fed into LAE_scanner. 

2. Once step 1 is completed and the independent person has given the final classifications, this notebook can be used to make a final list of LSDCat detections. This list will be used with ProFound to extract sources. 

This will take the LAE_Classifier outputs for the members in one group and will compare results


*For v15 We had 3 people to classify per field + a moderator. In v22, this is now 2 people + a moderator


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import glob
%matplotlib inline
%config Completer.use_jedi = False
pd.options.display.max_rows=500

# Select a galaxy

All cavaets of a notebook exsists. So it is receomdede to rest the kernal evertime a different galaxy is used. 

In [2]:
gal_name = 'MAGPI1505'

workdir = '/Users/wnanayakkara/Dropbox/MagPi/LSDCat_outputs_to_groups/v22'

# Compare between group members to produce a list for the final assesor

## First compare results between the group members 

Minor changes have been made to account for change in process between v15 --> v22

In [3]:
LSD_cat_catalogue = ascii.read(f'{workdir}/{gal_name}_LSDCat.cat',  
                              names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' ])



In [4]:
class_files = glob.glob(f'{workdir}/{gal_name}*class.dat')
print(class_files)
assert len(class_files)==2, 'there are more or less than 2 files'

['/Users/wnanayakkara/Dropbox/MagPi/LSDCat_classifications/v22/MAGPI1505_MAGPI1505_LSDCat_SSW_class.dat', '/Users/wnanayakkara/Dropbox/MagPi/LSDCat_classifications/v22/MAGPI1505_MAGPI1505_LSDCat_SBA_class.dat']


In [5]:
class_p1 = ascii.read(class_files[0], 
          names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' , 'Galaxy'] ).to_pandas().set_index('I')


class_p2 = ascii.read(class_files[1], 
          names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' , 'Galaxy'] ).to_pandas().set_index('I')

# class_p3 = ascii.read(class_files[2], 
#           names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' , 'Galaxy'] ).to_pandas().set_index('I')






In [6]:
all_classifications = pd.merge(class_p1, class_p2, left_index=True, right_index=True, 
                               how='outer', suffixes=('_p1', '_p2'))


# all_classifications = pd.merge(all_classifications, class_p3.add_suffix('_p3'), left_index=True, right_index=True, 
#                                 how='outer')

all_classification_grades =  all_classifications[['Galaxy_p1', 'Galaxy_p2']]#, 'Galaxy_p3']]


In [7]:
print("There are ", len(all_classification_grades), "candidates in total ")

284

In [8]:
## all common between the group
all_common = all_classification_grades.query('Galaxy_p1==Galaxy_p2')
print('There are ', len(all_common), ' common classifications')


# ## When there are 3 aggregators it is a bit complex
# all_common = all_classification_grades.query('Galaxy_p1==Galaxy_p2==Galaxy_p3')
# print('There are ', len(all_common), ' common classifications')

# diff_p3= all_classification_grades.query('(Galaxy_p1==Galaxy_p2) and  (Galaxy_p1!=Galaxy_p3)')
# print('p1==p2!=p3', len(diff_p3))

# diff_p1= all_classification_grades.query('(Galaxy_p3==Galaxy_p2) and  (Galaxy_p1!=Galaxy_p3)')
# print('p3==p2!=p1', len(diff_p1))

# diff_p2= all_classification_grades.query('(Galaxy_p3==Galaxy_p1) and  (Galaxy_p2!=Galaxy_p3)')
# print('p3==p1!=p2', len(diff_p2))



There are  177  common classifications


## Prepare a new file to input to LAE_scanner for sources with disagreements

If two classifiers have a common grade, we keep it.

However, if the common grade is 3, then these need to be sent to the final assesor. 
 
 

In [10]:
agreed = all_classification_grades.query('(Galaxy_p1==Galaxy_p2)')

### if 3 people
# agreed = all_classification_grades.query('(Galaxy_p3==Galaxy_p2)  | (Galaxy_p3==Galaxy_p1) | (Galaxy_p1==Galaxy_p2)')

print(len(agreed), ' sources are agreeable out of ', len(all_classification_grades))

agreed['agreed_grade'] = agreed.mode(axis=1)


print('There are ', len(agreed.query('agreed_grade==3')), 'source/s which need revisiting from the agreed list because they are classified as maybe')

177  sources are agreeable out of  284
There are  1 source/s which need revisiting from the agreed list because they are classified as maybe


/var/folders/r3/8rkwf3z12mgd39qrrfckpr3s0d32bj/T/ipykernel_4951/1573448613.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agreed['agreed_grade'] = agreed.mode(axis=1)


All other sources have two(three) different grades. They also need to be revisted by the group members and a common grade should be agreed upon. 

In [11]:
to_be_agreed = all_classification_grades[~all_classification_grades.index.isin(agreed.index)]

print(len(to_be_agreed), 'sources with two different classifications')

107 sources with two different classifications


lets add all that needs to be discussed together

In [12]:
to_be_discussed = all_classifications.reindex(to_be_agreed.index)
## add the maybe ones
to_be_discussed = pd.concat([to_be_discussed, all_classifications.reindex(agreed.query('agreed_grade==3').index)])

get the index based on LSD cat catalogue file. 
The retention of comments in the .cat file is important! 

In [13]:
LSD_inds = [np.where(index==LSD_cat_catalogue['I'])[0][0] for index in to_be_discussed.index  ]




use the file written below to consolidate the results in the group

In [14]:
LSD_cat_catalogue[LSD_inds].write(f'{workdir}/{gal_name}_combine_catalogue_to_be_consolidated.dat', 
                       format='ascii.fixed_width_no_header', delimiter=None, )




So basically, the above file should be run again with LAE_scanner and all sources to be agreed upon. 

Then technically, the newly classified sources should be combined with the previous list (and remove the duplicates)

# Make a final list of detected sources to be fed to Profound


## Reconcile with the group + assesor

In [6]:
### add a real file for this in future
class_reconciliation = ascii.read(f'{workdir}/{gal_name}_reassessment_ESW.dat', 
          names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' , 'Galaxy'] ).to_pandas().set_index('I')


### now we can now have maybe objects
assert len(class_reconciliation.query('Galaxy==3'))==0 , "There cannot be any MAYBE objects, all should be YES or NO"
class_reconciliation = class_reconciliation.rename(columns= {'Galaxy': 'agreed_grade'})

In [16]:
### make sure the total number of sources are preserved
sources_in_reconciliation_list =  len(class_reconciliation)

##agreed is everything except maybe
sources_agreed_on = len(all_classifications.reindex(agreed.query('agreed_grade!=3').index))

assert len(all_classifications) == sources_in_reconciliation_list + sources_agreed_on, "Source numbers don't match"


In [17]:
final_output = class_p1.reindex(agreed.query('agreed_grade==1').index)

final_output.drop('Galaxy', axis=1, inplace=True)
final_output['agreed_grade'] = agreed.query('agreed_grade==1').agreed_grade


final_output = pd.concat([final_output, class_reconciliation.query('agreed_grade==1')], axis=0)

## Check if any of the 'real' sources are duplicated

This means if there are multiple lines for a galaxy, LSDCat can identify those lines independently of each other. 
In this case, a judgement call should be made on which source to keep/remove. 
Ideally, the source with highest NPIX or highest S/N should be kept. 
It is always better to go to the cube and look in these situtations. 
Specially because Ly-alpha is extended so basing a judement on purely pixels vs S/N might not be best. 
I guess it doesn't really matter at the end because people who study halo properties would go to the NBs anyways. 



In [18]:

print('targets with multiple detections')
final_output[final_output.duplicated('ID')].ID



targets with multiple detections


I
120    84
67     45
Name: ID, dtype: int64

In [19]:
### manually put the IDS here 'ID==A| ID==B| ID==C' henceforth
final_output.query('ID==84| ID==45').sort_values('ID')

,ID,X_PEAK_SN,Y_PEAK_SN,Z_PEAK_SN,NPIX,DETSN_MAX,agreed_grade
I,,,,,,,
64,45,159.0,165.0,872.0,47,13.987032,1
67,45,159.0,165.0,2460.0,108,20.356070,1
119,84,132.0,329.0,1592.0,594,25.861885,1
120,84,132.0,329.0,1926.0,77,17.280361,1


In [28]:
# final_output.query('ID==164')

,ID,X_PEAK_SN,Y_PEAK_SN,Z_PEAK_SN,NPIX,DETSN_MAX,agreed_grade
I,,,,,,,
246,164,203,234,3543,307,45.761883,1
244,164,203,234,3330,20,13.356362,1


if the above is none, then we are good to go and provide the file for profound

**UPADTE 8/06/2021** This file (written below) not used for profound anymore. 
The file in the next section which has all columns from LSDCat_Measure will be used. 
This is because NB with the limits given by those extra columns in Profound. 

In [24]:
### NOT USED 
# final_output.to_csv(f'../LSDCat_classifications/{gal_name}_ELGs_final_output.dat', sep='\t')

### Perfrom any duplication checks with LAE_GUI 

If needed use the file generated below to interact with the duplicated sources. 

In [20]:
final_output.query('ID==84| ID==45').sort_values('ID').index.values

array([ 64,  67, 119, 120])

In [36]:

LSD_inds = final_output.query('ID==25| ID==118| ID==72').sort_values('ID').index.values
LSD_cat_catalogue[LSD_inds-1].write(f'{workdir}/{gal_name}_LSDCat_duplicate_checks.dat', 
                       format='ascii.fixed_width_no_header', delimiter=None, )





# Collate final detected sources with LSDCat Flux files to be fed into Profound
(from LSDCat Measurement Routine Version 1.51) 

In [37]:
from astropy.utils.diff import report_diff_values
import sys

In [38]:
### first sanity check to see if the files are the same
### this is because for the first five galaxies the larger files were created after the group assignments were made
### this is not necessary in future
gal_name = 'MAGPI1501'

LSD_cat_catalogue = ascii.read(f'{workdir}/{gal_name}_LSDCat.cat',  
                              names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' ])

LSD_cat_catalogue_new = ascii.read(f'{workdir}/{gal_name}_LSDCat.cat',  
                              names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' ])


identical = report_diff_values(LSD_cat_catalogue, LSD_cat_catalogue_new, fileobj=sys.stdout)


      I   ID X_PEAK_SN Y_PEAK_SN Z_PEAK_SN NPIX DETSN_MAX
     --- --- --------- --------- --------- ---- ---------
       1   1       312       138         0    8 11.882159
       2   2       275       154         0   73 21.465412
       3   3        69        43        14   13 12.381769
       4   3        68        43      2179   20 12.904837
       5   4        91        36        15    2 10.333988
       6   5       170       270        20  172 40.232159
       7   5       170       273       316    1  10.18404
       8   6       283       340        20  102 32.923962
       9   6       283       342       310  152  55.08073
      10   6       282       343       696  199 54.162987
     ... ...       ...       ...       ...  ...       ...
     310 208       231       131      3205    8 12.030111
     311 209       189        40      3228    3 10.412211
     312 210        94       367      3243    4 10.400422
     313 211       348        43      3251  190   24.5856
     314 212  

True

In [39]:
if not identical:
    print("ERROR: outputs are not identical")

In [40]:
LSD_cat_fluxes = Table.read(f'{workdir}/{gal_name}_LSDCat_fluxes.fits').to_pandas().set_index('I')



gal_detections = pd.read_csv(f'{workdir}/{gal_name}_ELGs_final_output.dat', sep='\t', index_col='I')



In [41]:
gal_combined = pd.merge(gal_detections, LSD_cat_fluxes, left_index=True, right_index=True, how='inner', 
         suffixes=('_lsdcat', '_lsdmeasure'))

In [42]:
cols_to_compare = ['ID', 'X_PEAK_SN','Y_PEAK_SN','Z_PEAK_SN','NPIX']


for cols in cols_to_compare:
    
    if not gal_combined[cols+'_lsdcat'].equals(gal_combined[cols+'_lsdmeasure']):
        print('values in', cols,  'columns dont match')
    
    
# 'DETSN_MAX' need to be compared seperately because it is a float

cols ='DETSN_MAX'
if len(gal_combined[~(np.round(gal_combined[cols+'_lsdcat'], 2) == np.round(gal_combined[cols+'_lsdmeasure'], 2))])!=0:
    print('there are unmatched values')
    
    

In [43]:
gal_combined.to_csv(f'{workdir}/{gal_name}_LSDCatFlux_detections_only.csv')

# LSDcat final results of all objects and noise 

This will create a final list of classifications for all LSDcat detections (real/maybe/noise). 

This is for record keeping purposes i.e. for a paper, proposals on LAE detection rates etc

In [46]:
LSD_cat_fluxes = Table.read(f'{workdir}/{gal_name}_LSDCat_fluxes.fits').to_pandas().set_index('I')

class_reconciliation = ascii.read(f'/Users/wnanayakkara/Dropbox/MagPi/LSDCat_classifications/{gal_name}_reassessment_TNA.dat', 
          names=['I' , 'ID' , 'X_PEAK_SN' , 'Y_PEAK_SN' , 'Z_PEAK_SN' , 'NPIX' , 'DETSN_MAX' , 'Galaxy'] ).to_pandas().set_index('I')


In [47]:
LSD_cat_fluxes.loc[agreed.query('agreed_grade!=3').index, 'agreed_grade'] = agreed.query('agreed_grade!=3').agreed_grade

LSD_cat_fluxes.loc[class_reconciliation.index, 'agreed_grade'] = class_reconciliation.Galaxy



In [48]:
LSD_cat_fluxes.query('agreed_grade!=agreed_grade')


,ID,X_PEAK_SN,Y_PEAK_SN,Z_PEAK_SN,NPIX,DETSN_MAX,X_SN,Y_SN,Z_SN,RA_SN,...,SIGMA_ISO,F_KRON,F_KRON_ERR,F_2KRON,F_2KRON_ERR,F_3KRON,F_3KRON_ERR,F_4KRON,F_4KRON_ERR,agreed_grade
I,,,,,,,,,,,,,,,,,,,,,


In [49]:
LSD_cat_fluxes.to_csv(f'{workdir}/{gal_name}_LSDCat_fluxes_and_final_grades.csv')
                      
                      
                      